In [136]:
%pip install torch==1.10.0 torchvision==0.11.1 torchtext==0.11.0 torchaudio==0.10.0 --quiet
%pip install pytorch-lightning==1.5.2 --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [137]:
# Importing libraries (p.24)
import pytorch_lightning as pl
import torch
from torch import nn, optim
from torch.autograd import Variable
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
from torchmetrics.functional import accuracy

In [138]:
# Check last 2 blocks ran fine
print("Torch version:", torch.__version__)
print("pytorch lightning version:", pl.__version__)

Torch version: 1.10.0+cpu
pytorch lightning version: 1.5.2


In [139]:
# Preparing the data (p.25)
# Define input features
xor_input = [Variable(torch.Tensor([0, 0])),
              Variable(torch.Tensor([0, 1])),
              Variable(torch.Tensor([1, 0])),
              Variable(torch.Tensor([1, 1]))]

In [140]:
# Define target variables
xor_target = [Variable(torch.Tensor([0])),
              Variable(torch.Tensor([1])),
              Variable(torch.Tensor([1])),
              Variable(torch.Tensor([0]))]

In [145]:
# Build dataset
xor_data = list(zip(xor_input, xor_target))
train_loader = DataLoader(xor_data, batch_size=1000)

In [146]:
# Configuring the model
class XORModel(pl.LightningModule):
  def __init__(self):

    super(XORModel,self).__init__()
    
    self.input_layer = nn.Linear(2, 4)
    self.output_layer = nn.Linear(4,1)

    self.sigmoid = nn.Sigmoid()

    self.loss = nn.MSELoss()

    num_workers = 8

  def forward(self, input):
    print("INPUT:", input.shape)
    x = self.input_layer(input)
    print("FIRST:", x.shape)
    x = self.sigmoid(x)
    print("SECOND:", x.shape)
    output = self.output_layer(x)
    print("THIRD:", output.shape)
    return output

  def configure_optimizers(self):
    params = self.parameters()
    optimizer = optim.Adam(params=params, lr = 0.01)
    return optimizer

  def training_step(self, batch, batch_idx):
    xor_input, xor_target = batch
    print("XOR INPUT:", xor_input.shape)
    print("XOR TARGET:", xor_target.shape)
    outputs = self(xor_input) 
    print("XOR OUTPUT:", outputs.shape)
    loss = self.loss(outputs, xor_target)
    return loss 

In [164]:
# Training the model (p.32)
from pytorch_lightning.utilities.types import TRAIN_DATALOADERS
checkpoint_callback = ModelCheckpoint()
model = XORModel()

trainer = pl.Trainer(max_epochs=500, callbacks=[checkpoint_callback])

trainer.fit(model, train_dataloaders=train_loader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name         | Type    | Params
-----------------------------------------
0 | input_layer  | Linear  | 12    
1 | output_layer | Linear  | 5     
2 | sigmoid      | Sigmoid | 0     
3 | loss         | MSELoss | 0     
-----------------------------------------
17        Trainable params
0         Non-trainable params
17        Total params
0.000     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

In [165]:
# Loading the model (p.34)
print(checkpoint_callback.best_model_path)
train_model = model.load_from_checkpoint(checkpoint_callback.best_model_path)

c:\Users\cavou\Desktop\DeepLearning-with-PyTorch-Lightning\Chapter02\lightning_logs\version_8\checkpoints\epoch=499-step=499.ckpt


In [166]:
# Making predictions (p.35)
test = torch.utils.data.DataLoader(xor_input, batch_size=1)
for val in xor_input:
  _ = train_model(val)
  print([int(val[0]),int(val[1])], int(_.round()))

[0, 0] 0
[0, 1] 1
[1, 0] 1
[1, 1] 0


In [167]:
print(checkpoint_callback.best_model_path)
train_model = model.load_from_checkpoint(checkpoint_callback.best_model_path)
total_accuracy = []
for xor_input, xor_target in train_loader:
  for i in range(100):
    output_tensor = train_model(xor_input)
    test_accuracy = accuracy(output_tensor, xor_target.int())
    total_accuracy.append(test_accuracy)
total_accuracy = torch.mean(torch.stack(total_accuracy))
print("TOTAL ACCURACY FOR 100 ITERATIONS: ", total_accuracy.item())

c:\Users\cavou\Desktop\DeepLearning-with-PyTorch-Lightning\Chapter02\lightning_logs\version_8\checkpoints\epoch=499-step=499.ckpt
TOTAL ACCURACY FOR 100 ITERATIONS:  1.0
